In [1]:
import pandas as pd
import folium
import geopandas
from shapely.geometry import Point

In [2]:
import json
import requests
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
coord_pairs = []
with open('data/population_lolland_4500.json') as json_file:
    data = json.load(json_file)
    for person in data['persons']:
        start_coord = person['activities'][0]['coord']
        end_coord = person['activities'][1]['coord']
        time = person['activities'][1]['start_time']
        szone = person['activities'][0]['zone']
        ezone = person['activities'][1]['zone']
        coord_pairs.append((start_coord[0],start_coord[1], end_coord[0], end_coord[1], time, szone, ezone))
        
outgoing = 0
incomming = 0
within = 0
sjobo = [360240, 360120, 360210, 360110, 360230, 360130, 360140, 360250]
for person in data['persons']:
    szone = int(person['activities'][0]['zone'])
    ezone = int(person['activities'][1]['zone'])
    
    if szone in sjobo and ezone in sjobo:
        within += 1
    elif szone in sjobo and ezone not in sjobo:
        outgoing += 1
    else:
        incomming += 1
        
print (outgoing, incomming, within)
print(outgoing + incomming + within)

683 743 3106
4532


In [3]:
od_raw = pd.read_csv('data/OD_LollandK_Nov19.csv')
od_raw.drop(index=[5624, 5627, 5644],axis=0, inplace=True)
lollandshp = geopandas.read_file('data/GIS/lolland/lolland.shp')
excludedshp = geopandas.read_file('data/GIS/lolland/excluded.shp')

In [4]:
od_raw.head()

,OutputQueryRunID,StartStop,DestinationsStop,StartKommune,DestinationsKommune,RejseStartTidsbånd,AntalRejsende,FraStopNr,TilStopNr,FraLTML2ZoneID,TilLTML2ZoneID,FraKomNr,TilKomNr
0,20928,Nykøbing F St.,Nakskov St.,Guldborgsund,Lolland,0,3.410988,8600824,8602623,376140,360240,376,360
1,20928,Nykøbing F St.,Maribo St.,Guldborgsund,Lolland,0,1.306749,8600824,8602619,376140,360120,376,360
2,20928,Nykøbing F St.,Søllested St.,Guldborgsund,Lolland,0,0.125949,8600824,8602645,376140,360210,376,360
3,20928,Øster Toreby St.,Nakskov St.,Guldborgsund,Lolland,0,0.043589,8602604,8602623,376210,360240,376,360
4,20928,Øster Toreby St.,Maribo St.,Guldborgsund,Lolland,0,0.023714,8602604,8602619,376210,360120,376,360


In [5]:
od_raw['AntalRejsende'].sum()

4525.079759179847

In [6]:
od_raw['AntalRejsende'].sum()*21

95026.6749427768

In [7]:
stops = pd.read_csv('data/stops.txt')
stops = stops.set_index('stop_id')
stops.head()

,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,platform_code
stop_id,,,,,,,,,
461011300,NaN,Dyrupgårds Alle (Odense Kommune),NaN,55.354918,10.345035,0,NaN,NaN,NaN
8600718,NaN,Vipperød St.,NaN,55.666828,11.738710,0,NaN,NaN,NaN
8600719,NaN,Holbæk St.,NaN,55.715750,11.708598,0,NaN,NaN,NaN
8600716,NaN,Hvalsø St.,NaN,55.595537,11.859821,0,NaN,NaN,NaN
8600717,NaN,Tølløse St.,NaN,55.611893,11.772803,0,NaN,NaN,NaN


# Plot od with polylines

In [9]:
od_7 = od_raw[od_raw['RejseStartTidsbånd']==6]
od_7.head()
# od_7 = od_raw

,OutputQueryRunID,StartStop,DestinationsStop,StartKommune,DestinationsKommune,RejseStartTidsbånd,AntalRejsende,FraStopNr,TilStopNr,FraLTML2ZoneID,TilLTML2ZoneID,FraKomNr,TilKomNr
2068,20928,Maribo St.,Nakskov St.,Lolland,Lolland,6,25.093692,8602619,8602623,360120,360240,360,360
2069,20928,Nakskov St.,Nykøbing F St.,Lolland,Guldborgsund,6,23.250763,8602623,8600824,360240,376140,360,376
2070,20928,Maribo St.,Nykøbing F St.,Lolland,Guldborgsund,6,19.722943,8602619,8600824,360120,376140,360,376
2071,20928,Nakskov St.,Maribo St.,Lolland,Lolland,6,13.391053,8602623,8602619,360240,360120,360,360
2072,20928,Søllested St.,Nakskov St.,Lolland,Lolland,6,13.125570,8602645,8602623,360210,360240,360,360


In [10]:
line = []
no_location = set()
for path in od_7.iterrows():
    try:
        lat = stops.loc[path[1]['FraStopNr']]['stop_lat']
        lon = stops.loc[path[1]['FraStopNr']]['stop_lon']
        slatlon = (lat,lon)
    except:
        no_location = no_location.union({path[1]['FraStopNr']})
        continue
    try:
        lat = stops.loc[path[1]['TilStopNr']]['stop_lat']
        lon = stops.loc[path[1]['TilStopNr']]['stop_lon']
        elatlon = (lat,lon)
    except:
        no_location = no_location.union({path[1]['TilStopNr']})
        continue      
    line.append((slatlon,elatlon,path[1]['AntalRejsende']))

In [11]:
line[0]

((54.776641266147, 11.501672983703001),
 (54.833248872169, 11.138192792463),
 25.093691855668997)

In [19]:
no_location

{2160, 38700, 44685, 45116, 53265, 1000004, 1000005}

In [20]:
od_7[(od_7['FraStopNr'].isin(no_location)) | (od_7['TilStopNr'].isin(no_location))]

,OutputQueryRunID,StartStop,DestinationsStop,StartKommune,DestinationsKommune,RejseStartTidsbånd,AntalRejsende,FraStopNr,TilStopNr,FraLTML2ZoneID,TilLTML2ZoneID,FraKomNr,TilKomNr
2394,20928,Kastrup (Kastrupvej),Nakskov St.,Vordingborg,Lolland,6,0.176965,45116,8602623,390140,360240,390,360
2609,20928,Tog fra Sverige,Nakskov St.,Portzoner,Lolland,6,0.100215,1000005,8602623,912421,360240,-1,360
2695,20928,Tog fra Sverige,Maribo St.,Portzoner,Lolland,6,0.085194,1000005,8602619,912421,360120,-1,360
3019,20928,Maribo St.,Tog mod Sverige,Lolland,Portzoner,6,0.050545,8602619,1000004,360120,912421,360,-1
3227,20928,Nakskov St.,Tog mod Sverige,Lolland,Portzoner,6,0.039905,8602623,1000004,360240,912421,360,-1
3283,20928,Maribo St.,Teglholmsgade (Teglholm Allé),Lolland,København,6,0.037165,8602619,53265,360120,102850,360,101
3458,20928,Tog fra Sverige,Søllested St.,Portzoner,Lolland,6,0.029959,1000005,8602645,912421,360210,-1,360
3556,20928,Hagelundsvej (Årbyesvej),Nørrebro St. (Nørrebrogade),Lolland,København,6,0.026558,45849,2160,360140,102450,360,101
3985,20928,Søllested St.,Tog mod Sverige,Lolland,Portzoner,6,0.015783,8602645,1000004,360210,912421,360,-1
4690,20928,Stationsvej (Bygaden),"Slagelse Sygehus, Midt (Fælledvej)",Lolland,Slagelse,6,0.006532,40201,38700,360250,330130,360,330


In [21]:
COORD = [55.63135, 13.70622]
m = folium.Map(location=COORD, zoom_start=8)

for l in line:
#     folium.PolyLine(l[0:2], weight=max(l[2],0.3), tooltip=l[2]).add_to(m)
    folium.PolyLine(l[0:2], weight=min(1,l[2]*100), tooltip=l[2]).add_to(m)
m.save('data/demand.html')

In [13]:
od_7['AntalRejsende'].sum()*21

12341.41068873892

# What is LTML2 zones?

In [12]:
od_raw.head()

,OutputQueryRunID,StartStop,DestinationsStop,StartKommune,DestinationsKommune,RejseStartTidsbånd,AntalRejsende,FraStopNr,TilStopNr,FraLTML2ZoneID,TilLTML2ZoneID,FraKomNr,TilKomNr
0,20928,Nykøbing F St.,Nakskov St.,Guldborgsund,Lolland,0,3.410988,8600824,8602623,376140,360240,376,360
1,20928,Nykøbing F St.,Maribo St.,Guldborgsund,Lolland,0,1.306749,8600824,8602619,376140,360120,376,360
2,20928,Nykøbing F St.,Søllested St.,Guldborgsund,Lolland,0,0.125949,8600824,8602645,376140,360210,376,360
3,20928,Øster Toreby St.,Nakskov St.,Guldborgsund,Lolland,0,0.043589,8602604,8602623,376210,360240,376,360
4,20928,Øster Toreby St.,Maribo St.,Guldborgsund,Lolland,0,0.023714,8602604,8602619,376210,360120,376,360


In [13]:
point = []
for path in od_raw.drop_duplicates(subset=['FraStopNr']).iterrows():
    try:
        lat = stops.loc[path[1]['FraStopNr']]['stop_lat']
        lon = stops.loc[path[1]['FraStopNr']]['stop_lon']
        text = path[1]['FraLTML2ZoneID']
        slatlon = (lat,lon)
    except:
        continue   
    point.append((slatlon,text))

In [14]:
COORD = [55.63135, 13.70622]
m = folium.Map(location=COORD, zoom_start=8)

for l in point:
    folium.Marker(l[0],
                 icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: 'black'">{l[1]}</div>""")
                 ).add_to(m)


In [15]:
m.save('data/demand.html')

# zones in lolland

In [14]:
zones = set()
for path in od_raw.drop_duplicates(subset=['FraStopNr']).iterrows():
    try:
        lat = stops.loc[path[1]['FraStopNr']]['stop_lat']
        lon = stops.loc[path[1]['FraStopNr']]['stop_lon']
        text = path[1]['FraLTML2ZoneID']
    except:
        continue   
    slatlon = Point(lon,lat)
    if lollandshp.contains(slatlon).any():
        zones = zones.union({text})

In [15]:
zones

{360110, 360120, 360130, 360140, 360210, 360230, 360240, 360250}

# Filter excluded (big cities)

In [27]:
line = []
no_location = set()
filtered = []
for path in od_raw.iterrows():
    try:
        lat = stops.loc[path[1]['FraStopNr']]['stop_lat']
        lon = stops.loc[path[1]['FraStopNr']]['stop_lon']
        slatlon = Point(lon,lat)
    except:
        no_location = no_location.union({path[1]['FraStopNr']})
        continue
    try:
        lat = stops.loc[path[1]['TilStopNr']]['stop_lat']
        lon = stops.loc[path[1]['TilStopNr']]['stop_lon']
        elatlon = Point(lon,lat)
    except:
        no_location = no_location.union({path[1]['TilStopNr']})
        continue
    
    if (excludedshp.contains(slatlon).any() or excludedshp.contains(elatlon).any()) and \
       (not (lollandshp.contains(slatlon).any()) or not (lollandshp.contains(elatlon).any())):
        continue
        
    if excludedshp.contains(slatlon).any() and excludedshp.contains(elatlon).any():
        continue
        
    filtered.append(path)

In [28]:
f = [fil[1] for fil in filtered]

In [29]:
filtered_od = pd.DataFrame(f)

In [ ]:
line = []
no_location = set()
for path in filtered_od.iterrows():
    if path[1]['RejseStartTidsbånd']!=7:
        continue
    try:
        lat = stops.loc[path[1]['FraStopNr']]['stop_lat']
        lon = stops.loc[path[1]['FraStopNr']]['stop_lon']
        slatlon = (lat,lon)
    except:
        no_location = no_location.union({path[1]['FraStopNr']})
        continue
    try:
        lat = stops.loc[path[1]['TilStopNr']]['stop_lat']
        lon = stops.loc[path[1]['TilStopNr']]['stop_lon']
        elatlon = (lat,lon)
    except:
        no_location = no_location.union({path[1]['TilStopNr']})
        continue      
    line.append((slatlon,elatlon,path[1]['AntalRejsende'], path[1].name))

In [ ]:
COORD = [55.63135, 13.70622]
m = folium.Map(location=COORD, zoom_start=8)

for l in line:
    folium.PolyLine(l[0:2], weight=max(l[2],0.3), tooltip=l[3]).add_to(m)
m.save('data/demand.html')

In [30]:
filtered_od['AntalRejsende'].sum()*21

23616.39432758221

In [ ]:
od_7 = filtered_od[filtered_od['RejseStartTidsbånd']==7]

In [ ]:
od_7['AntalRejsende'].sort_values()

In [23]:
filtered_od.to_csv('data/OD_lolland_filtered.csv')